# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [61]:
import torch as th

In [62]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [63]:
y = x + x

In [64]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [65]:
import syft as sy

In [66]:
hook = sy.TorchHook(th)

W0705 16:02:44.548796 11660 hook.py:97] Torch was already hooked... skipping hooking process


In [67]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [68]:
bob = sy.VirtualWorker(hook, id="bob")

In [69]:
bob._objects

{}

In [70]:
x = th.tensor([1,2,3,4,5])

In [71]:
x = x.send(bob)

In [72]:
bob._objects

{52559349317: tensor([1, 2, 3, 4, 5])}

In [73]:
x.location

<VirtualWorker id:bob #objects:1>

In [74]:
x.id_at_location

52559349317

In [75]:
x.id

63499809022

In [76]:
x.owner

<VirtualWorker id:me #objects:0>

In [77]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [78]:
x

(Wrapper)>[PointerTensor | me:63499809022 -> bob:52559349317]

In [79]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [80]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [81]:
# try this project here!

In [82]:
alice = sy.VirtualWorker(hook, id = "alice")

In [83]:
def send_tensor_to_workers(tensor, workers =  []):
    y = tensor.send(*workers)
    return  y

def get_tensor(tensor):
    y = tensor.get()
    return y
    

In [84]:
y = th.Tensor([10,2,3,4])
workers = [bob,alice]
y = send_tensor_to_workers(y, workers =workers )

for worker in workers:
    print("Objects of worker {} :{} ".format(worker.id, worker._objects))

y = get_tensor(y)

print("")
print("Tensor:{}".format(y))

Objects of worker bob :{16403660884: tensor([10.,  2.,  3.,  4.])} 
Objects of worker alice :{86618182561: tensor([10.,  2.,  3.,  4.])} 

Tensor:[tensor([10.,  2.,  3.,  4.]), tensor([10.,  2.,  3.,  4.])]


# Lesson: Introducing Remote Arithmetic

In [85]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [86]:
x

(Wrapper)>[PointerTensor | me:564890580 -> bob:88034512379]

In [87]:
y

(Wrapper)>[PointerTensor | me:57169692518 -> bob:92059012416]

In [88]:
z = x + y

In [89]:
z

(Wrapper)>[PointerTensor | me:62261077189 -> bob:52033151247]

In [90]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [91]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:64254112051 -> bob:29376037834]

In [92]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [93]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [94]:
z = (x + y).sum()


In [95]:
z.backward()

(Wrapper)>[PointerTensor | me:40925180237 -> bob:14769626311]

In [96]:
x = x.get()

In [97]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [98]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [99]:
# try this project here!

In [100]:
import torch as th
from torch.autograd import Variable
X =th.tensor([[1.0,1.0],[2.0,1.0],[3.0,1.0],[4.0,1.0],[5.0,1.0],[6.0,1.0]]
             ,requires_grad = True )


y = th.tensor([[2.0],[4.0],[6.0],[8.0],[10.0],[12.0]]
              ,requires_grad = True)



(X*y).shape

torch.Size([6, 2])

### Send Tensors to Bob

In [101]:
X = X.send(bob)
y = y.send(bob)

## Model Definition

In [102]:
class LinearRegression():
    
    def __init__(self):
        
        #Can initialize weigths with zeros, not on neural nets.
        self.w = th.tensor( [[0.0],[0.0]], requires_grad = True) 
        
        #send to bob
        self.w =  self.w.send(bob)
        
        
            
    def forward(self,x):
        y = th.matmul(x,self.w)
        
        return y
    
    def backward(self, loss):
        """
        loss: torch Tensor
        recieves outouput of linear model
        return gradient with respect to weights
        """
     
        loss.backward()
        
        
    def gradient_step(self,learning_rate):
        
        assert self.w.grad  is not None,"Compute gradients first."
        
        self.w.data.sub_(learning_rate * self.w.grad)
        
        self.w.grad *= 0
        

        
    def l2_norm(self,y,y_pred):
        """
        y:torch tensor
            targets
        y_pred:torch tensor
            predictions make by model
        compute and return l-2 norm between y and y_pred:
        """
        
        m = y.shape[0]
        diff = y - y_pred
        
        
        return th.matmul(diff.t(), diff)/m

In [103]:
model = LinearRegression()

In [104]:


for _ in range(25):
    
    
    y_pred = model.forward(X)
    
    #print("y_pred",y_pred.clone().get())
    

    #Compute cost function
    loss = model.l2_norm(y, y_pred)
    print("Loss", loss.clone().get().data.item())
   
    #Compute gradients of loss with respect to parameters w
    model.backward(loss)


    #print("Gradients:")
    #print("for w:")

    #w_grad = model.w.grad.clone().get()
    #print(w_grad)

    #print("for b:")
    #b_grad = model.b.grad.clone().get()
    #print(b_grad)
    
    
    #Gradient descent step
    model.gradient_step(learning_rate = 0.05)
    
       
    
#.get() params causes to left bob. Use .clone() to make a copy
print("Parameters w: {}".format(model.w.clone().get()))


Loss 60.66666793823242
Loss 21.747970581054688
Loss 7.810120105743408
Loss 2.8180999755859375
Loss 1.029659390449524
Loss 0.38846683502197266
Loss 0.15813873708248138
Loss 0.07496897131204605
Loss 0.04452216997742653
Loss 0.032979678362607956
Loss 0.028230242431163788
Loss 0.025935737416148186
Loss 0.024541640654206276
Loss 0.02349088154733181
Loss 0.022582896053791046
Loss 0.02174530178308487
Loss 0.020951509475708008
Loss 0.020191175863146782
Loss 0.019460072740912437
Loss 0.01875598169863224
Loss 0.018077662214636803
Loss 0.01742393523454666
Loss 0.01679382286965847
Loss 0.0161865446716547
Loss 0.015601209364831448
Parameters w: tensor([[1.9347],
        [0.2795]], requires_grad=True)


In [105]:
x_ =th.tensor([[3.0,1.0],[5.0,1.0],[20.0,1.0]])
print(x_.shape)
x_ = x_.send(bob)

y = model.forward( x_  )
y.clone().get()

torch.Size([3, 2])


tensor([[ 6.0837],
        [ 9.9531],
        [38.9737]], requires_grad=True)

# Lesson: Garbage Collection and Common Errors


In [106]:
bob = bob.clear_objects()

In [107]:
bob._objects

{}

In [108]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [109]:
bob._objects

{75193012304: tensor([1, 2, 3, 4, 5])}

In [110]:
del x

In [111]:
bob._objects

{}

In [112]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [113]:
bob._objects

{98387425159: tensor([1, 2, 3, 4, 5])}

In [114]:
x = "asdf"

In [115]:
bob._objects

{}

In [116]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [117]:
x

(Wrapper)>[PointerTensor | me:85170261023 -> bob:73279110367]

In [118]:
bob._objects

{73279110367: tensor([1, 2, 3, 4, 5])}

In [119]:
x = "asdf"

In [120]:
bob._objects

{73279110367: tensor([1, 2, 3, 4, 5])}

In [121]:
del x

In [122]:
bob._objects

{73279110367: tensor([1, 2, 3, 4, 5])}

In [123]:
bob = bob.clear_objects()
bob._objects

{}

In [124]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [125]:
bob._objects

{48767363911: tensor([1, 2, 3, 4, 5])}

In [126]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [127]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:62587176306 -> bob:85370813755]
Tensor B: tensor([1, 1, 1, 1, 1])

In [128]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [129]:
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [130]:
from torch import nn, optim

In [131]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [132]:
# A Toy Model
model = nn.Linear(2,1)

In [133]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [134]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(11.4219)
tensor(1.3498)
tensor(0.4276)
tensor(0.2505)
tensor(0.1659)
tensor(0.1123)
tensor(0.0766)
tensor(0.0527)
tensor(0.0365)
tensor(0.0255)
tensor(0.0180)
tensor(0.0128)
tensor(0.0091)
tensor(0.0066)
tensor(0.0048)
tensor(0.0035)
tensor(0.0026)
tensor(0.0019)
tensor(0.0014)
tensor(0.0010)


In [135]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [136]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [137]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [138]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    print("Model", model.parameters())
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [139]:
train()

Model <generator object Module.parameters at 0x0000022E8BCFC2B0>
tensor(1.4446, requires_grad=True)
tensor(0.1269, requires_grad=True)
tensor(0.1395, requires_grad=True)
tensor(0.0945, requires_grad=True)
tensor(0.0827, requires_grad=True)
tensor(0.0630, requires_grad=True)
tensor(0.0549, requires_grad=True)
tensor(0.0426, requires_grad=True)
tensor(0.0372, requires_grad=True)
tensor(0.0295, requires_grad=True)
tensor(0.0255, requires_grad=True)
tensor(0.0208, requires_grad=True)
tensor(0.0177, requires_grad=True)
tensor(0.0150, requires_grad=True)
tensor(0.0125, requires_grad=True)
tensor(0.0109, requires_grad=True)
tensor(0.0088, requires_grad=True)
tensor(0.0080, requires_grad=True)
tensor(0.0063, requires_grad=True)
tensor(0.0060, requires_grad=True)
tensor(0.0046, requires_grad=True)
tensor(0.0045, requires_grad=True)
tensor(0.0033, requires_grad=True)
tensor(0.0033, requires_grad=True)
tensor(0.0024, requires_grad=True)
tensor(0.0025, requires_grad=True)
tensor(0.0018, requires_g

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [1]:
bob.clear_objects()
alice.clear_objects()

NameError: name 'bob' is not defined

In [2]:
x = th.tensor([1,2,3,4,5]).send(bob)

NameError: name 'th' is not defined

In [3]:
x = x.send(alice)

NameError: name 'x' is not defined

In [4]:
bob._objects

NameError: name 'bob' is not defined

In [5]:
alice._objects

NameError: name 'alice' is not defined

In [6]:
y = x + x

NameError: name 'x' is not defined

In [7]:
y

NameError: name 'y' is not defined

In [8]:
bob._objects

NameError: name 'bob' is not defined

In [9]:
alice._objects

NameError: name 'alice' is not defined

In [10]:
jon = sy.VirtualWorker(hook, id="jon")

NameError: name 'sy' is not defined

In [11]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

NameError: name 'bob' is not defined

In [12]:
bob._objects

NameError: name 'bob' is not defined

In [13]:
alice._objects

NameError: name 'alice' is not defined

In [14]:
x = x.get()
x

NameError: name 'x' is not defined

In [15]:
bob._objects

NameError: name 'bob' is not defined

In [16]:
alice._objects

NameError: name 'alice' is not defined

In [17]:
x = x.get()
x

NameError: name 'x' is not defined

In [18]:
bob._objects

NameError: name 'bob' is not defined

In [19]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

NameError: name 'bob' is not defined

In [20]:
bob._objects

NameError: name 'bob' is not defined

In [21]:
alice._objects

NameError: name 'alice' is not defined

In [22]:
del x

NameError: name 'x' is not defined

In [23]:
bob._objects

NameError: name 'bob' is not defined

In [24]:
alice._objects

NameError: name 'alice' is not defined

# Lesson: Pointer Chain Operations

In [25]:
bob.clear_objects()
alice.clear_objects()

NameError: name 'bob' is not defined

In [26]:
x = th.tensor([1,2,3,4,5]).send(bob)

NameError: name 'th' is not defined

In [27]:
bob._objects

NameError: name 'bob' is not defined

In [28]:
alice._objects

NameError: name 'alice' is not defined

In [29]:
x.move(alice)

NameError: name 'x' is not defined

In [30]:
bob._objects

NameError: name 'bob' is not defined

In [170]:
alice._objects

{18118267155: tensor([1, 2, 3, 4, 5])}

In [171]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [172]:
bob._objects

{29913377643: tensor([1, 2, 3, 4, 5])}

In [173]:
alice._objects

{18118267155: tensor([1, 2, 3, 4, 5]),
 83513707602: (Wrapper)>[PointerTensor | alice:83513707602 -> bob:29913377643]}

In [174]:
x.remote_get()

(Wrapper)>[PointerTensor | me:95037513910 -> alice:83513707602]

In [175]:
bob._objects

{}

In [176]:
alice._objects

{18118267155: tensor([1, 2, 3, 4, 5]), 83513707602: tensor([1, 2, 3, 4, 5])}

In [177]:
x.move(bob)

(Wrapper)>[PointerTensor | me:95037513910 -> bob:95037513910]

In [178]:
x

(Wrapper)>[PointerTensor | me:95037513910 -> bob:95037513910]

In [179]:
bob._objects

{95037513910: tensor([1, 2, 3, 4, 5])}

In [180]:
alice._objects

{18118267155: tensor([1, 2, 3, 4, 5])}

# Final Project

- Central server is not trusted.
- One worker will aggregate gradients (take averge).
- Then, send model to central server.

In [174]:
import syft as sy
import torch as th 


In [175]:
hook = sy.TorchHook(th)


W0706 16:55:03.691732  5548 hook.py:97] Torch was already hooked... skipping hooking process


In [176]:
from torch import  nn,optim

In [177]:
# A Toy Dataset
data = th.tensor([[1.0,1.0],[3,1.0],[1,1.0],[0,1.0],[23.0,1.0],[6,1.0],[7,1.0],[15,1.0],[9.0,1.0]], requires_grad=True)


target = th.tensor([[2.],[6.0], [2.0], [0],[46.0],[12.0],[14.0],[30.0],[18.0]], requires_grad=True)
        


In [178]:
m = 3 #number of workers

In [179]:
workers  =[sy.VirtualWorker(hook, id = "w"+ str(i)) for i in range(m)]
chunk_size = data.shape[0]//m



## Make datasets

In [180]:

#make a mini-dataset, one per worker
datasets = [ [
              data[j*chunk_size : (j+1) * chunk_size  ] ,
              target[j*chunk_size : (j+1) * chunk_size] 
             ]
            
             for  j in range(m)
           ]

print("Datasets[0]", *datasets[0], sep= "\n")

#send datasets to workers
for k in range(m):
    
    datasets[k][0] = datasets[k][0].send(workers[k])
    
    datasets[k][1] = datasets[k][1].send(workers[k])
    
    
       

Datasets[0]
tensor([[1., 1.],
        [3., 1.],
        [1., 1.]], grad_fn=<SliceBackward>)
tensor([[2.],
        [6.],
        [2.]], grad_fn=<SliceBackward>)


In [181]:
import random
from random import shuffle

def train_secure_grads(m,datasets,workers, iterations=20):
    """
    iterations: int
    
    m: int
        number of models/workers
        
    datasets: list of lists of tensors
        [ [data_1,targets_1],[data_2, targets_2], ...     ]
    
    workers: list of VirtualWorkers
    
    iterations: int 
        number of SGD steps to do.
        
    """

    # Create a model for each worker 
    models = [ nn.Linear(2,1) for _ in range(m)]
    
    
    # Global model
    global_model = nn.Linear(2,1)
    
    # Create optims 
    optims = []

    for i,t in enumerate(datasets):
        
        _data,_target  = t[0],t[1]
        
        
        # Data preparation
        
        # send model to the data
        models[i] = models[i].send(_data.location)
        
        #create optimizer for model i 
        #learning rate >= 0.1, makess trainig diverge on most models with little data.
        optim_i = optim.SGD(params=models[i].parameters(), lr=0.001)
        optims.extend( [optim_i] )
        
        for _ in range(iterations):            
            # do normal training
            optims[i].zero_grad()
            pred = models[i](_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            
            optims[i].step()

            
            print("Loss: {} for model: {}".format( loss.clone().get().item(), i ))
        
        print("Params {} for model: {}".format(models[i].weight.clone().get() ,i))
        print()
    
    
    # Average models on selected worker
    
    #shuffle models. Select first to aggregate gradients
    shuffle(models)
    
    # Selected worker to aggregate gradients.
    sel_model = models[0]
    
    #move_model = lambda x: x.move(sel_worker)

    w,b = sel_model.weight.data ,sel_model.bias.data
    
    for model in models[1:]:
        # Move model to selected worker
        model.move(sel_model.location)
        
        w+= model.weight.data
        b+=  model.bias.data
        
        # Set weight
        #sel_model.weight.data.set_(
            
        #    sel_model.weight.data  +  model.weight.data)     
        
        # Set bias
        
        #sel_model.bias.data.set_(
            
        #     sel_model.bias.data  +  model.bias.data)     
        
    
    w = w/m
    b = b/m
    #sel_model.weight.data.set_( w/m )
    
    #sel_model.bias.data.set_( b/m )
    
    with th.no_grad():
    
        global_model.weight.set_(  w.get() #sel_model.weight.data.clone().get()
                                  )
        global_model.bias.set_(    b.get() #sel_model.bias.data.clone().get()
                                  )
    
    print("Params for model global model: {}".format(global_model.weight.clone()))

        
    return global_model


In [182]:
train_secure_grads(m = 3, datasets = datasets,workers = workers, iterations = 50  )

Loss: 32.099464416503906 for model: 0
Loss: 30.09716796875 for model: 0
Loss: 28.220884323120117 for model: 0
Loss: 26.462696075439453 for model: 0
Loss: 24.815147399902344 for model: 0
Loss: 23.271282196044922 for model: 0
Loss: 21.824565887451172 for model: 0
Loss: 20.46888542175293 for model: 0
Loss: 19.198505401611328 for model: 0
Loss: 18.008052825927734 for model: 0
Loss: 16.89249610900879 for model: 0
Loss: 15.84711742401123 for model: 0
Loss: 14.867501258850098 for model: 0
Loss: 13.949501991271973 for model: 0
Loss: 13.089242935180664 for model: 0
Loss: 12.283084869384766 for model: 0
Loss: 11.52762508392334 for model: 0
Loss: 10.819665908813477 for model: 0
Loss: 10.15622615814209 for model: 0
Loss: 9.534486770629883 for model: 0
Loss: 8.951836585998535 for model: 0
Loss: 8.405805587768555 for model: 0
Loss: 7.894091606140137 for model: 0
Loss: 7.414533615112305 for model: 0
Loss: 6.965103626251221 for model: 0
Loss: 6.543907165527344 for model: 0
Loss: 6.149170875549316 for 

Linear(in_features=2, out_features=1, bias=True)

1.6740666666666666